<a href="https://colab.research.google.com/github/Depersonalizc/AL-cancer-detection/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CSC4080 Midterm Project
## 📝Active Learning for Label-Efficient Histopathologic Cancer Detection


*Group 24* 

- 🐷 CHEN Ang   (118010009)

- 🐸 FU Quanzhi (118010070)








In [ ]:
print("fuck world")

hello world


In [1]:
print('ca says hello world')

ca says hello world
